In [22]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os, sys 
sys.path.append('..')
from collections import deque
import copy
import itertools
import aoc_utils as au
import math 
from tqdm import tqdm
import itertools
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
input_text = au.read_txt_file_lines()
n_rows = len(input_text)
codes = input_text
print(codes)

['169A', '279A', '540A', '869A', '789A']


## THoughts
- Use Dijkstra to find shortest path(s) between each of the 11-keypad buttons. 
- Use Dijkstra to find shortest path(s) between A and any of the 4 arrows on 5-key button.
- I think you can solve the problems layer by layer .. 
- Roll out .. 

In [24]:

class Graph:
    def __init__(self, nodes, paths):
        self.nodes = nodes
        self.paths = paths
        n = len(self.nodes)
        self.adj = np.zeros((n, n), dtype=int)
        for i1, n1 in enumerate(self.nodes):
            for i2, n2 in enumerate(self.nodes):
                if n1 + n2 in self.paths:
                    self.adj[i1, i2] = 1
                    self.adj[i2, i1] = 1

        self.get_all_paths()
        
    def get_all_paths(self):
        self.all_paths = {}
        for i1, n1 in enumerate(self.nodes):
            for i2, n2 in enumerate(self.nodes):
                if n1 + n2 in self.paths:
                    self.all_paths[n1 + n2] = [self.paths[n1 + n2]]
                else:  # get ALL shortest paths
                    self.all_paths[n1 + n2] = self.get_shortest_paths(i1, i2)

    def get_shortest_paths(self, i1, i2):
        '''BFS'''
        shortest_paths = []
        n_nod = len(self.nodes)

        dist = [np.inf] * n_nod
        dist[i1] = 0

        queue = deque([(i1, [i1])])

        while len(queue) > 0:
            node, p = queue.popleft()
            for neigh in np.where(self.adj[node, :])[0]:
                if neigh == i2:
                    if dist[node] + 1 <= dist[neigh]:
                        shortest_paths.append((neigh, p + [neigh]))
                        dist[neigh] = dist[node] + 1
                elif node == i2:
                    pass
                elif dist[neigh] >= dist[node] + 1:
                    dist[neigh] = dist[node] + 1
                    if len(p) > dist[i2]:
                        pass
                    else:
                        queue.append((neigh, p + [neigh]))

        shortest_paths = [x[1] for x in shortest_paths]
        new_paths = []
        for p in shortest_paths:
            p_new = []
            for ii in range(len(p) - 1):
                tmp = self.nodes[p[ii]] + self.nodes[p[ii + 1]]
                p_new.append(self.paths[tmp])
            p_new = ''.join(p_new)
            new_paths.append(p_new)
        return new_paths

    def is_grouped(self, s):
        # Remove consecutive duplicates and count unique groups
        unique_groups = ''.join(char for i, char in enumerate(s) if i == 0 or s[i] != s[i-1])
        return len(set(unique_groups)) == len(unique_groups)
    
class KeyPad(Graph):
    def __init__(self):

        self.paths = {}
        for hor in ['12', '23', '45', '56', '78', '89', '0A']:
            self.paths[hor] = '>'
            self.paths[hor[::-1]] = '<'

        for ver in ['41', '52', '63', '74', '85', '96', '20', '3A']:
            self.paths[ver] = 'v'
            self.paths[ver[::-1]] = '^'

        super().__init__(nodes=list('0123456789A'), paths=self.paths)

        ## get rid of all inefficient paths (<v< instead of <<v or v<<)
        for k, v in self.all_paths.items():
            self.all_paths[k] = [p for p in v if self.is_grouped(p)]

class Direct(Graph):
    def __init__(self):

        self.paths = {}
        for hor in ['^A', '<v', 'v>']:
            self.paths[hor] = '>'
            self.paths[hor[::-1]] = '<'

        for ver in ['^v', 'A>']:
            self.paths[ver] = 'v'
            self.paths[ver[::-1]] = '^'

        super().__init__(nodes=list('<v>^A'), paths=self.paths)
        
        ## get rid of inefficient paths:
        self.all_paths['<A'] = ['>>^']
        self.all_paths['A<'] = ['v<<']
        self.all_paths['Av'] = ['v<']
        self.all_paths['^>'] = ['>v']
        self.all_paths['>^'] = ['^<']
        self.all_paths['vA'] = ['>^']


In [25]:
kp = KeyPad()
dr = Direct()

In [26]:
dict_bp = {}
sums = []
n_operators = 6  # number of robots (excl human)

def insert_a_presses(c_paths):
    ''' ['>>', '^v>'] becomes ['>>', 'A', '^v>', 'A']'''
    result = [] 
    for c in c_paths:
        tmp = []
        for action in c:
            tmp.append(action)
            tmp.append('A')

        result.append(''.join(tmp))
    
    return result

for c in codes:
    num_c = int(''.join([x for x in c if x != 'A']))
    c_paths = []
    dict_bp['kp'] = c
    c = 'A' + c
    for ii in range(len(c) - 1):
        c_combo = c[ii] + c[ii + 1]
        c_paths.append(kp.all_paths[c_combo])

    c_paths = list(itertools.product(*c_paths))  # (path between A and 1st, path between 1st and 2nd, ..)
    c_paths = insert_a_presses(c_paths)
    dict_bp[0] = c_paths  ## these are now the instructions for r1.
    
    ## have to do this twice more for r2 and h
    for i_operator in tqdm(range(1, n_operators + 1)):
        op_paths = []
        for combo in dict_bp[i_operator - 1]:  # previous paths
            tmp_path = []
            combo = 'A' + combo  # start at A position 
            for ii in range(len(combo) - 1):
                d_combo = combo[ii] + combo[ii + 1]
                tmp_path.append(dr.all_paths[d_combo])
            tmp_path = [x if len(x) > 0 else [''] for x in tmp_path]
            tmp_path = list(itertools.product(*tmp_path))
            tmp_path = insert_a_presses(tmp_path)
            for p in tmp_path:
                op_paths.append(p)

        ## greedy but seems to work: only keep shortest paths at each stage. 
        lens = [len(x) for x in op_paths]
        min_len = min(lens)
        lens_keep = [True if l == min_len else False for l in lens]
        # assert sum(lens_keep) == len(lens_keep), (sum(lens_keep), len(lens_keep))
        dict_bp[i_operator] = [x for ix, x in enumerate(op_paths) if lens_keep[ix]]
        # dict_bp[i_operator - 1] = None  # no longer needed
        
    len_c = min([len(x) for x in dict_bp[i_operator]])  # final min len
    print(len_c, num_c)
    print(f'code {c} has {len(dict_bp[i_operator])} combis')
    # print(dict_bp[2])
    sums.append(len_c * num_c)

print(sums)
print(sum(sums))


100%|██████████| 6/6 [00:00<00:00, 7933.74it/s]


3160 169
code A169A has 1 combis


100%|██████████| 6/6 [00:00<00:00, 7133.17it/s]


3146 279
code A279A has 1 combis


100%|██████████| 6/6 [00:00<00:00, 5922.76it/s]


3138 540
code A540A has 1 combis


100%|██████████| 6/6 [00:00<00:00, 3146.12it/s]


2988 869
code A869A has 1 combis


100%|██████████| 6/6 [00:00<00:00, 2650.15it/s]

2792 789
code A789A has 1 combis
[534040, 877734, 1694520, 2596572, 2202888]
7905754


In [20]:
print(dict_bp['kp'])
print(dict_bp[0])
print(dict_bp[1])
print(dict_bp[2])

789A
['^^^<<A>A>AvvvA']
['<AAAv<AA>>^AvA^AvA^Av<AAA>^A']
['v<<A>>^AAAv<A<A>>^AAvAA^<A>Av<A>^A<A>Av<A>^A<A>Av<A<A>>^AAAvA^<A>A']


In [21]:
dict_bp[6]

['v<A<A>>^Av<<A>>^AvAA^<A>Av<A<AA>>^AvAA^<A>Av<A>^AA<Av<A>>^AvA^Av<A<A>>^Av<<A>>^AAvAA^<A>Av<A>^AA<Av<A>>^AvA^AAv<A<A>>^AvA^<A>AAv<<A>>^Av<A<A>>^AvAA^<A>Av<A>^A<A>Av<A<A>>^Av<<A>>^AvAA^<A>Av<A>^A<Av<A>>^AvA^AAv<A<AA>>^AvAA^<A>Av<A<A>>^Av<<A>>^AvAA^<A>Av<A>^AA<Av<A>>^AvA^Av<A<A>>^AvA^<A>Av<<A>>^AvA^AAAv<A<A>>^Av<<A>>^AvAA^<A>Av<A<AA>>^AvAA^<A>Av<A>^AA<Av<A>>^AvA^Av<A<A>>^Av<<A>>^AAvAA^<A>Av<A>^AA<Av<A>>^AvA^Av<A<A>>^AvA^<A>AAv<<A>>^Av<A<A>>^AvAA^<A>Av<A>^A<A>Av<A<A>>^Av<<A>>^AvAA^<A>Av<A<AA>>^AvAA^<A>AAv<A>^AA<Av<A>>^AvA^Av<A<A>>^AvA^<A>AAv<<A>>^Av<A<A>>^AvAA^<A>Av<A>^A<A>Av<A<A>>^Av<<A>>^AvAA^<A>Av<A>^A<Av<A>>^AvA^AAv<A<AA>>^AvAA^<A>Av<A<A>>^Av<<A>>^AvAA^<A>Av<A>^AA<Av<A>>^AvA^Av<A<A>>^AvA^<A>Av<<A>>^AvA^AAv<A<A>>^Av<<A>>^AvAA^<A>Av<A<AA>>^AvAA^<A>Av<A>^AA<Av<A>>^AvA^Av<A<A>>^AvA^<A>Av<<A>>^Av<A<A>>^AvAA^<A>Av<A>^A<A>AAv<A<A>>^Av<<A>>^AAvAA^<A>Av<A>^AA<Av<A>>^AvA^Av<A<A>>^Av<<A>>^AvAA^<A>Av<A<AA>>^AvAA^<A>Av<A>^AA<Av<A>>^AvA^Av<A<A>>^AvA^<A>AAv<<A>>^Av<A<A>>^AvAA^<A>Av<A>^A<A>Av<A<A>>^

## Part 2:

Have to make use of fact that it's just 1/few flips between seqs.. 


In [10]:
tmp = ['v<<A>>^Av<A<A>>^AAvAA^<A>Av<A>^AA<Av<A>>^AvA^Av<<A>>^AvA^Av<A<A>>^AAAvA^<A>A', 
       'v<<A>>^Av<A<A>>^AAvAA^<A>Av<<A>>^AvA<A>^AA<A>Av<<A>>^AvA^Av<A<A>>^AAAvA^<A>A']

assert len(tmp[0]) == len(tmp[1])

for i_s, s1 in enumerate(tmp[0]):
    s2 = tmp[1][i_s]
    if s1 != s2:
        print(i_s, s1, s2) 

28 A <
29 > A
30 ^ >
31 A >
32 A ^
33 < A
34 A v
35 v A
39 > ^
40 ^ A
42 v <
44 ^ >
